<a href="https://colab.research.google.com/github/OdysseusPolymetis/ariane_classes/blob/main/BERT_multilingualSimilarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <center>**Mesures de similarités entre le grec et les traductions latines**</center>

Requirements

In [ ]:
!pip install torch transformers numpy stanza umap-learn sentencepiece

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!git clone https://github.com/PerseusDL/canonical-greekLit.git

Cloning into 'canonical-greekLit'...
remote: Enumerating objects: 59342, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 59342 (delta 14), reused 50 (delta 14), pack-reused 59292
Receiving objects: 100% (59342/59342), 513.64 MiB | 25.92 MiB/s, done.
Resolving deltas: 100% (39515/39515), done.
Updating files: 100% (2634/2634), done.


In [ ]:
!git clone https://github.com/PerseusDL/canonical-latinLit.git

Les imports

In [ ]:
import torch
import glob
from joblib import Parallel, delayed
import pickle
from collections import defaultdict
import os
import numpy as np
import re
from lxml import etree
import stanza
from tqdm import tqdm
import unicodedata

Je récupère le modèle finetuné et je l'applique au corpus général.

In [ ]:
import torch
from transformers import pipeline
from transformers import BertTokenizer, BertModel
pipe = pipeline("fill-mask", model="UGARIT/grc-alignment")
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("UGARIT/grc-alignment")
model = AutoModelForMaskedLM.from_pretrained("UGARIT/grc-alignment")
model.eval()  # Mode évaluation

Some weights of the model checkpoint at UGARIT/grc-alignment were not used when initializing XLMRobertaForMaskedLM: ['psi_cls.decoder.weight', 'psi_cls.bias', 'psi_cls.transform.bias', 'psi_cls.transform.weight']
- This IS expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at UGARIT/grc-alignment were not used when initializing XLMRobertaForMaskedLM: ['psi_cls.decoder.weight', 'psi_cls.bias', 'psi_cls.transform.bias', 'psi_cls.transform.weight']
- This IS expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model t

XLMRobertaForMaskedLM(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
  

In [ ]:
print(tokenizer.tokenize("un exemple avec accent ὡἤ"))

['▁un', '▁exemple', '▁avec', '▁accent', '▁', 'ὡ', 'ἤ']


parcours des xml

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

XLMRobertaForMaskedLM(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
  

In [ ]:
def extract_body_content_from_xml(file_path):
    parser = etree.XMLParser(recover=True)
    tree = etree.parse(file_path, parser)
    nsmap = tree.getroot().nsmap
    default_ns = nsmap.get(None)

    if default_ns:
        body = tree.find(".//ns:body", namespaces={"ns": default_ns})
    else:
        body = tree.find(".//body")

    if body is None:
        raise ValueError(f"No <body> element found in {file_path}")

    return etree.tostring(body, method="text", encoding="unicode")

def is_latin(filename):
    return re.search(r'lat\d+\.xml$', filename) is not None

def is_greek(filename):
    return re.search(r'grc\d+\.xml$', filename) is not None

def extract_texts_from_directory(directory_path):
    """
    Extrait les textes de chaque fichier XML dans le répertoire spécifié et les stocke dans un dictionnaire.
    """
    texts = {}
    for root, _, files in os.walk(directory_path):
        for file in files:
            if is_latin(file) or is_greek(file):
                file_path = os.path.join(root, file)
                content = extract_body_content_from_xml(file_path)
                texts[file] = content
    return texts

In [ ]:
def tokenize_and_encode_texts(texts, tokenizer):
    tokenized_texts = {}
    for text_key, text_value in texts.items():
        tokens = tokenizer.tokenize(text_value)
        # Tronquer à la longueur maximale de 512
        tokens = tokens[:512]
        input_ids = tokenizer.convert_tokens_to_ids(tokens)
        tokenized_texts[text_key] = {
            'tokens': tokens,
            'input_ids': input_ids
        }
    return tokenized_texts



In [ ]:
greek_corpus = extract_texts_from_directory('/content/canonical-greekLit')

In [ ]:
latin_corpus = extract_texts_from_directory('/content/canonical-latinLit')

In [ ]:
premiere_cle, premiere_valeur = list(latin_corpus.items())[0]
print(premiere_cle, premiere_valeur)

In [ ]:
stanza.download('grc')

INFO:stanza:Downloading default packages for language: grc (Ancient_Greek) ...


INFO:stanza:Finished downloading models and saved to /root/stanza_resources.


In [ ]:
stanza.download('la')

INFO:stanza:Downloading default packages for language: la (Latin) ...


INFO:stanza:Finished downloading models and saved to /root/stanza_resources.


In [ ]:
def process_text(text, nlp):
    """
    Traite le texte avec Stanza pour obtenir à la fois les formes fléchies et les lemmes.
    """
    doc = nlp(text)

    processed_text = {
        "lemmas": [],
        "forms": []
    }

    for sentence in doc.sentences:
        lemmatized_sentence = []
        forms_sentence = []
        for word in sentence.words:
            lemmatized_sentence.append(word.lemma)
            forms_sentence.append(word.text)
        processed_text["lemmas"].append(lemmatized_sentence)
        processed_text["forms"].append(forms_sentence)

    return processed_text


In [ ]:
nlp = stanza.Pipeline('grc', processors='tokenize,pos,lemma', use_gpu=True)

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: grc (Ancient_Greek):
| Processor | Package         |
-------------------------------
| tokenize  | proiel          |
| pos       | proiel_nocharlm |
| lemma     | proiel_nocharlm |

INFO:stanza:Using device: cuda
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Done loading processors!


In [ ]:
nlp_latin = stanza.Pipeline('la', processors='tokenize,pos,lemma', use_gpu=True)

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: la (Latin):
| Processor | Package       |
-----------------------------
| tokenize  | ittb          |
| pos       | ittb_nocharlm |
| lemma     | ittb_nocharlm |

INFO:stanza:Using device: cuda
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Done loading processors!


In [ ]:
import random

# 1. Choisir aléatoirement 10% des fichiers
filenames = list(greek_corpus.keys())
sample_size = int(0.1 * len(filenames))
sample_filenames = random.sample(filenames, sample_size)

# 2. Effectuer la lemmatisation sur cette portion
sample_corpus_processed = {}
for filename in tqdm(sample_filenames, desc="Processing"):
    sample_corpus_processed[filename] = process_text(greek_corpus[filename], nlp)

Processing: 100%|██████████| 81/81 [04:54<00:00,  3.64s/it]


In [ ]:
filenames = list(greek_corpus.keys())

# 2. Effectuer la lemmatisation sur cette portion
greek_corpus_processed = {}
for filename in tqdm(filenames, desc="Processing"):
    greek_corpus_processed[filename] = process_text(greek_corpus[filename], nlp)

Processing: 100%|██████████| 812/812 [1:07:23<00:00,  4.98s/it]


In [ ]:
filenames = list(latin_corpus.keys())

# 2. Effectuer la lemmatisation sur cette portion
latin_corpus_processed = {}
for filename in tqdm(filenames, desc="Processing"):
    latin_corpus_processed[filename] = process_text(latin_corpus[filename], nlp_latin)

Processing: 100%|██████████| 427/427 [58:33<00:00,  8.23s/it]


In [ ]:
with open('/content/drive/MyDrive/embeddings_save/lemmatized_greek_perseus.pkl', 'wb') as file:
    pickle.dump(greek_corpus_processed, file)

In [ ]:
with open('/content/drive/MyDrive/embeddings_save/greek_corpus_processed.pkl', 'rb') as file:
    greek_corpus_processed = pickle.load(file)

In [ ]:
with open('/content/drive/MyDrive/embeddings_save/latin_corpus_processed.pkl', 'wb') as file:
    pickle.dump(latin_corpus_processed, file)

In [ ]:
with open('/content/drive/MyDrive/embeddings_save/latin_corpus_processed.pkl', 'rb') as file:
    latin_corpus_processed = pickle.load(file)

In [ ]:
for key, value in list(latin_corpus_processed.items())[:5]:
    lemmas = value["lemmas"]
    forms = value["forms"]
    print(f"Filename: {key}")
    print(f"Lemmatized Content: {' '.join(lemmas[0])[:200]}...")  # Print only first 200 chars of the lemmatized content of the first sentence
    print(f"Forms Content: {' '.join(forms[0])[:200]}...\n")  # Print only first 200 chars of the forms content of the first sentence

In [ ]:
forms_test = set(form for doc in sample_corpus_processed.values() for sentence in doc["forms"] for form in sentence)

In [ ]:
print(len(forms_test))

143901


In [ ]:
def split_long_sentences(corpus, tokenizer, max_length=100):
    split_corpus = {"forms": {}, "lemmas": {}}

    for filename, text in tqdm(corpus.items(), desc="Splitting long sentences"):
        split_forms, split_lemmas = [], []

        for forms_sentence, lemmas_sentence in zip(text["forms"], text["lemmas"]):
            tokenized_words = [tokenizer.tokenize(word) for word in forms_sentence]
            token_lengths = [len(tokens) for tokens in tokenized_words]

            sub_sentence_tokens, sub_forms, sub_lemmas = [], [], []
            for word, lemma, tokens, token_length in zip(forms_sentence, lemmas_sentence, tokenized_words, token_lengths):
                if len(sub_sentence_tokens) + token_length <= max_length:
                    sub_forms.append(word)
                    sub_lemmas.append(lemma)
                    sub_sentence_tokens.extend(tokens)
                else:
                    split_forms.append(sub_forms)
                    split_lemmas.append(sub_lemmas)
                    sub_forms, sub_lemmas, sub_sentence_tokens = [word], [lemma], tokens
            if sub_forms:
                split_forms.append(sub_forms)
                split_lemmas.append(sub_lemmas)

        split_corpus["forms"][filename] = split_forms
        split_corpus["lemmas"][filename] = split_lemmas

    return split_corpus



In [ ]:
def tokenize_sentence(sentence, tokenizer):
    return tokenizer.tokenize(sentence)

In [ ]:
split_greek_corpus = split_long_sentences(greek_corpus_processed, tokenizer, 100)
#split_latin_corpus = split_long_sentences(latin_corpus_lemmatized, tokenizer)

Splitting long sentences: 100%|██████████| 812/812 [07:53<00:00,  1.72it/s]


In [ ]:
split_latin_corpus = split_long_sentences(latin_corpus_processed, tokenizer, 100)

Splitting long sentences: 100%|██████████| 427/427 [06:26<00:00,  1.11it/s]


In [ ]:
with open('/content/drive/MyDrive/embeddings_save/split_latin_corpus_100.pkl', 'rb') as file:
    split_latin_corpus = pickle.load(file)
with open('/content/drive/MyDrive/embeddings_save/split_greek_corpus_100.pkl', 'rb') as file:
    split_greek_corpus = pickle.load(file)

In [ ]:
for key, value in list(split_greek_corpus["forms"].items())[:5]:
    print(f"Filename: {key}")
    print(f"Forms Content (first sentence): {' '.join(value[0])[:200]}...\n")  # Print only first 200 chars of the forms content of the first sentence

print("\n---\n")

for key, value in list(split_greek_corpus["lemmas"].items())[:5]:
    print(f"Filename: {key}")
    print(f"Lemmatized Content (first sentence): {' '.join(value[0])[:200]}...\n")  # Print only first 200 chars of the lemmatized content of the first sentence

In [ ]:
def verify_split_corpus(split_corpus, tokenizer, max_length=510):
    # Ici, nous utilisons tqdm pour envelopper l'itération sur les fichiers
    for filename in tqdm(split_corpus["forms"].keys(), desc="Verifying files"):
        forms_sentences = split_corpus["forms"][filename]
        lemmas_sentences = split_corpus["lemmas"][filename]

        if len(forms_sentences) != len(lemmas_sentences):
            print(f"Error: Mismatched number of sentences for file {filename}")
            continue

        for idx, (forms_sentence, lemmas_sentence) in enumerate(zip(forms_sentences, lemmas_sentences)):
            tokenized_sentence = tokenizer.tokenize(" ".join(forms_sentence))

            if len(tokenized_sentence) > max_length:
                print(f"Error: Sentence {idx} in file {filename} exceeds {max_length} tokens.")

            if len(forms_sentence) != len(lemmas_sentence):
                print(f"Error: Mismatched number of words in sentence {idx} of file {filename}")

In [ ]:
print("Starting verification...")
verify_split_corpus(split_greek_corpus, tokenizer)
print("\nVerification complete!")

Starting verification...


Verifying files: 100%|██████████| 812/812 [06:02<00:00,  2.24it/s]


Verification complete!


In [ ]:
# Endroit où stocker tous les auteurs disponibles
grc_target_authors = ["tlg0059","tlg0086","tlg1325", "tlg0626","tlg1304","tlg0632","tlg0591","tlg0593","tlg1562","tlg1705","tlg0014","tlg0610"]
lat_target_authors = ["phi0474", "phi1017","stoa0255","phi1014","tlg0557","phi0550","tlg0628","tlg0562","phi1254","phi1002","stoa0058","phi0684","phi1212"]

In [ ]:
#Première tranche latine
grc_target_authors = ["tlg0059","tlg0086","tlg1325", "tlg0626","tlg1304","tlg0632","tlg0591","tlg0593","tlg1562","tlg1705","tlg0014","tlg0610"]
lat_target_authors = ["phi0474", "phi1017","stoa0255","phi1014","tlg0557","phi0684"]

In [ ]:
#Seconde tranche latine
grc_target_authors = ["tlg0059","tlg0086","tlg1325", "tlg0626","tlg1304","tlg0632","tlg0591","tlg0593","tlg1562","tlg1705","tlg0014","tlg0610"]
lat_target_authors = ["phi0550", "tlg0628", "tlg0562", "phi1254", "phi1002", "stoa0058", "phi1212"]

In [ ]:
filtered_lat_corpus = {}
filtered_grc_corpus = {}
for categorie in split_latin_corpus:
    filtered_lat_corpus[categorie] = {fichier: phrases for fichier, phrases in split_latin_corpus[categorie].items() if any(substring in fichier for substring in lat_target_authors)}
for categorie in split_greek_corpus:
    filtered_grc_corpus[categorie] = {fichier: phrases for fichier, phrases in split_greek_corpus[categorie].items() if any(substring in fichier for substring in grc_target_authors)}

In [ ]:
num_entries_to_show = 1  # Changer cette valeur selon le nombre d'entrées que vous souhaitez voir

for categorie, contenu in filtered_lat_corpus.items():
    print(f"Catégorie: {categorie}")

    counter = 0
    for fichier, phrases in contenu.items():
        print(f"Nom du fichier: {fichier}")
        print(phrases)

        counter += 1
        if counter >= num_entries_to_show:
            break

    print("-" * 40)

In [ ]:
import multiprocessing

def tokenize_batch(args):
    batch_sentences_forms, batch_sentences_lemmas, tokenizer = args
    # Convertir les listes de mots en phrases pour le tokénisateur
    batch_sentences = [" ".join(sentence) for sentence in batch_sentences_forms]

    encoding = tokenizer(batch_sentences, truncation=True, padding='longest', return_tensors="pt", max_length=100)

    batch_tokens = []
    batch_origin_indices = []
    batch_subtoken_lemmas = [] # <-- Liste pour stocker les lemmes associés aux subtokens

    for sent_forms, sent_lemmas in zip(batch_sentences_forms, batch_sentences_lemmas):
        tokens = []
        origin_indices = []
        subtoken_lemmas = []  # <-- Liste temporaire pour cette phrase
        for idx, (form, lemma) in enumerate(zip(sent_forms, sent_lemmas)):
            word_tokens = tokenizer.tokenize(form)
            tokens.extend(word_tokens)
            origin_indices.extend([idx] * len(word_tokens))
            subtoken_lemmas.extend([lemma] * len(word_tokens))  # <-- Associez chaque subtoken au lemme
        batch_tokens.append(tokens)
        batch_origin_indices.append(origin_indices)
        batch_subtoken_lemmas.append(subtoken_lemmas)  # <-- Ajoutez la liste temporaire à la liste globale

    return encoding["input_ids"].tolist(), encoding["attention_mask"].tolist(), batch_sentences, batch_tokens, batch_origin_indices, batch_subtoken_lemmas  # <-- Ajoutez batch_subtoken_lemmas à la sortie

def batched_tokenization(corpus, tokenizer, batch_size):
    all_input_ids = []
    all_attention_masks = []
    all_sentence_tokens = []
    all_origin_indices = []
    all_subtoken_lemmas = []  # Nouvelle liste pour stocker les lemmes associés aux subtokens

    # Génération d'une liste de toutes les phrases du corpus pour les formes et les lemmes
    sentences_forms = [sentence for text in corpus["forms"].values() for sentence in text]
    sentences_lemmas = [sentence for text in corpus["lemmas"].values() for sentence in text]

    # Assurez-vous que les formes et les lemmes sont alignés
    assert len(sentences_forms) == len(sentences_lemmas), "Mismatch between forms and lemmas sentences."

    # Préparation des batches de phrases
    batches = [(sentences_forms[i:i+batch_size], sentences_lemmas[i:i+batch_size], tokenizer) for i in range(0, len(sentences_forms), batch_size)]

    with multiprocessing.Pool(processes=multiprocessing.cpu_count()) as pool:
        for input_ids, attention_masks, batch_sents, batch_tokens, batch_origin_inds, batch_lemmas in tqdm(pool.imap_unordered(func=tokenize_batch, iterable=batches), total=len(batches), desc="Tokenizing sentences"):
            all_input_ids.extend(input_ids)
            all_attention_masks.extend(attention_masks)
            all_sentence_tokens.extend(batch_tokens)
            all_origin_indices.extend(batch_origin_inds)
            all_subtoken_lemmas.extend(batch_lemmas)  # Ajoutez les lemmes associés aux subtokens à la liste globale

    return all_input_ids, all_attention_masks, all_sentence_tokens, all_origin_indices, all_subtoken_lemmas  # Retournez all_subtoken_lemmas



In [ ]:
greek_all_input_ids, greek_all_attention_masks, greek_all_sentences_tokens, greek_all_origin_indices, greek_all_subtoken_lemmas = batched_tokenization(filtered_grc_corpus, tokenizer, batch_size=512)

Tokenizing sentences: 100%|██████████| 177/177 [00:47<00:00,  3.74it/s]


In [ ]:
latin_all_input_ids, latin_all_attention_masks, latin_all_sentences_tokens, latin_all_origin_indices, latin_all_subtoken_lemmas = batched_tokenization(filtered_lat_corpus, tokenizer, batch_size=512)

Tokenizing sentences: 100%|██████████| 63/63 [00:19<00:00,  3.27it/s]


In [ ]:
print("Sample input IDs:", greek_all_input_ids[:5])  # Affiche les 5 premiers éléments
print("Sample attention masks:", greek_all_attention_masks[:5])  # Affiche les 5 premiers éléments
print("Sample sentence tokens:", greek_all_sentences_tokens[:5])  # Affiche les 5 premiers éléments
print("Sample sentence tokens:", greek_all_subtoken_lemmas[:5])

Sample input IDs: [[101, 479, 10781, 12649, 58317, 468, 15233, 17198, 31712, 480, 13140, 17762, 484, 31712, 472, 21263, 31712, 10649, 465, 14669, 29613, 467, 99509, 23788, 12649, 465, 31625, 53428, 19491, 44306, 10484, 483, 14669, 87276, 58281, 12649, 469, 29223, 12526, 10487, 480, 19582, 29223, 14669, 27393, 19038, 12649, 10356, 475, 15233, 70076, 15860, 117, 468, 12526, 110568, 16099, 479, 14669, 63444, 485, 34359, 16146, 35790, 51650, 480, 22360, 15751, 484, 31712, 465, 14669, 70076, 117, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 10487, 484, 21263, 465, 43140, 62913, 33947, 10358, 10487, 474, 13140, 20660, 12649, 10358, 10487, 471, 84236, 14669, 117, 480, 60846, 15233, 27835, 10484, 102, 0, 0, 

In [ ]:
!gdown 1-ArRYmyJSMRjhjoZSKy7rpYZWwOvHtkx
!gdown 1-QBR9py7M6Md4nUOKpbdDBZEZw8lwujA
!gdown 1-GU2O9ONXMBffjR3Rga1kvG_Lx-ehf81
!gdown 1-DA0ablAwlEnL74OB_CfZaEMIuto3Gxp
!gdown 1-L4TR4hoeA6k-ICHljInEAnmX0pjmsOU

!gdown 1-s8f3A1q-OrrngiYLnlQoUuW-YFstOXb
!gdown 1-YGFNFdKJi18z7EcwfVBxuWY4du27NcM
!gdown 1-alBaz4kPMHgCaXYUADXdOC6x1Tzj1bP
!gdown 1-rUO6ZPBdCkSLx9FzKZOrDwL3haDcQc8
!gdown 1-ovYsEBXIrnA7hu5VSNnwwTnplFVLxHx

In [ ]:
with open("/content/drive/MyDrive/embeddings_save/greek_all_input_ids_100.pkl", "rb") as f:
    greek_all_input_ids = pickle.load(f)
with open("/content/drive/MyDrive/embeddings_save/greek_attention_masks_100.pkl", "rb") as f:
    greek_all_attention_masks = pickle.load(f)
with open("/content/drive/MyDrive/embeddings_save/greek_all_sentences_tokens_100.pkl", "rb") as f:
    greek_all_sentences_tokens = pickle.load(f)
with open("/content/drive/MyDrive/embeddings_save/greek_all_origin_indices_100.pkl", "rb") as f:
    greek_all_origin_indices = pickle.load(f)
with open("/content/drive/MyDrive/embeddings_save/greek_all_subtoken_lemmas_100.pkl", "rb") as f:
    greek_all_subtoken_lemmas = pickle.load(f)

In [ ]:
with open("/content/drive/MyDrive/embeddings_save/latin_all_input_ids_100.pkl", "rb") as f:
    latin_all_input_ids = pickle.load(f)
with open("/content/drive/MyDrive/embeddings_save/latin_attention_masks_100.pkl", "rb") as f:
    latin_all_attention_masks = pickle.load(f)
with open("/content/drive/MyDrive/embeddings_save/latin_all_sentences_tokens_100.pkl", "rb") as f:
    latin_all_sentences_tokens = pickle.load(f)
with open("/content/drive/MyDrive/embeddings_save/latin_all_origin_indices_100.pkl", "rb") as f:
    latin_all_origin_indices = pickle.load(f)
with open("/content/drive/MyDrive/embeddings_save/latin_all_subtoken_lemmas_100.pkl", "rb") as f:
    latin_all_subtoken_lemmas = pickle.load(f)

In [ ]:
import numpy as np

def compute_weights(length):
    # Création d'un array avec les positions des tokens
    positions = np.arange(length)

    # Calcul de la position du milieu
    mid_position = length / 2.0

    # Paramètre de mise à l'échelle pour contrôler la largeur de la fonction gaussienne
    sigma = length / 6.0

    # Calcul des poids en utilisant une fonction gaussienne
    weights = np.exp(-(positions - mid_position)**2 / (2 * sigma**2))

    # Normalisation des poids pour qu'ils somment à 1
    weights /= weights.sum()

    return weights


In [ ]:
words_to_check = ['Ἀρχίαδος', 'ἀποκηρύσσω', 'περιλείματ']

model = model.to('cuda')

# Tokenize et obtenez les embeddings pour chaque mot
word_embeddings = {}
for word in words_to_check:
    inputs = tokenizer(word, return_tensors="pt", padding='max_length', truncation=True, max_length=10)
    inputs = {key: value.to('cuda') for key, value in inputs.items()}
    with torch.no_grad():
      outputs = model(**inputs, output_hidden_states=True)
      last_hidden_state = outputs.hidden_states[-1]
      word_embeddings[word] = last_hidden_state.squeeze(0)

In [ ]:
# Afficher les embeddings pour chaque token de chaque mot
for word, embeddings in word_embeddings.items():
    print(f"Embeddings pour le mot '{word}':")
    tokens = tokenizer.tokenize(word)
    for token, embed in zip(tokens, embeddings):
        print(f"Token: {token}, Embedding: {embed[:5]}...")  # affiche seulement les 5 premières dimensions pour la lisibilité
    print("\n")

In [ ]:
def get_contextual_embeddings_without_weights(processed_texts, model, tokenizer, device, all_input_ids, all_attention_masks, all_origin_indices, all_sentences_tokens, all_subtoken_lemmas, batch_size=32):
    model.eval()
    lemma_embeddings_sum = {}
    lemma_token_counts = {}

    current_idx = 0
    for filename, forms_sentences in tqdm(processed_texts["forms"].items(), desc="Processing files"):
        lemmas_sentences = processed_texts["lemmas"][filename]

        for forms_sentence, lemmas_sentence in zip(forms_sentences, lemmas_sentences):
            assert len(forms_sentence) == len(lemmas_sentence), f"Forms and lemmas length mismatch in file {filename}."

            # Assume that all_input_ids, all_attention_masks, and all_origin_indices have the same structure
            input_ids = all_input_ids[current_idx]
            attention_masks = all_attention_masks[current_idx]
            origin_indices = all_origin_indices[current_idx]
            subtoken_lemmas = all_subtoken_lemmas[current_idx]  # Lemmas pour les subtokens actuels

            input_tensor = torch.tensor(input_ids).unsqueeze(0).to(device)
            attention_masks_tensor = torch.tensor(attention_masks).unsqueeze(0).to(device)

            with torch.no_grad():
                  outputs = model(input_tensor, attention_mask=attention_masks_tensor)
                  hidden_states = outputs.last_hidden_state[0]  # Prenez le premier batch (devrait être la seule phrase)

            # Pas de logique de poids ici

            for token_idx, (token, lemma) in enumerate(zip(all_sentences_tokens[current_idx], subtoken_lemmas)):
                # Utilisez le lemme directement car il est déjà aligné avec les subtokens
                if lemma not in lemma_embeddings_sum:
                    lemma_embeddings_sum[lemma] = np.zeros(hidden_states.shape[1], dtype=np.float32)
                    lemma_token_counts[lemma] = 0

                lemma_embeddings_sum[lemma] += hidden_states[token_idx].cpu().numpy()  # Simplement ajouter l'embedding
                lemma_token_counts[lemma] += 1  # Comptez simplement le nombre d'occurrences

            current_idx += 1

    averaged_embeddings = {lemma: lemma_embeddings_sum[lemma] / lemma_token_counts[lemma] for lemma in lemma_embeddings_sum.keys()}
    return averaged_embeddings

In [ ]:
from torch.nn.utils.rnn import pad_sequence
from transformers import AutoTokenizer, AutoModel

def get_contextual_embeddings_with_weights(processed_texts, model, tokenizer, device, all_input_ids, all_attention_masks, all_origin_indices, all_sentences_tokens, all_subtoken_lemmas, batch_size=32):

    lemma_embeddings_sum = {}
    lemma_token_counts = {}

    # Total number of sentences
    total_sentences = len(all_input_ids)

    for start_idx in tqdm(range(0, total_sentences, batch_size), desc="Processing batches"):
        end_idx = min(start_idx + batch_size, total_sentences)

        # Extract batched data
        batch_input_ids = all_input_ids[start_idx:end_idx]
        batch_attention_masks = all_attention_masks[start_idx:end_idx]
        batch_sentences_tokens = all_sentences_tokens[start_idx:end_idx]
        batch_subtoken_lemmas = all_subtoken_lemmas[start_idx:end_idx]

        # Padding sequences within the batch to have the same length
        input_tensor = pad_sequence([torch.tensor(seq) for seq in batch_input_ids], batch_first=True).to(device)
        attention_masks_tensor = pad_sequence([torch.tensor(mask) for mask in batch_attention_masks], batch_first=True).to(device)

        with torch.no_grad():
            outputs = model(input_tensor, attention_mask=attention_masks_tensor, output_hidden_states=True)
            batch_hidden_states = outputs.hidden_states[-1]  # take the last hidden state

        # Iterate over individual sentence embeddings in the batch
        for idx, hidden_states in enumerate(batch_hidden_states):
            weights = compute_weights(len(batch_sentences_tokens[idx]))

            for token_idx, (token, lemma) in enumerate(zip(batch_sentences_tokens[idx], batch_subtoken_lemmas[idx])):
                if lemma not in lemma_embeddings_sum:
                    lemma_embeddings_sum[lemma] = np.zeros(hidden_states.shape[1], dtype=np.float32)
                    lemma_token_counts[lemma] = 0

                lemma_embeddings_sum[lemma] += weights[token_idx] * hidden_states[token_idx].cpu().numpy()
                lemma_token_counts[lemma] += weights[token_idx]

    averaged_embeddings = {lemma: lemma_embeddings_sum[lemma] / lemma_token_counts[lemma] for lemma in lemma_embeddings_sum.keys()}
    return averaged_embeddings



In [ ]:
philo_greek_embeddings = get_contextual_embeddings_with_weights(
    processed_texts=filtered_grc_corpus,
    model=model,
    tokenizer=tokenizer,
    device=device,
    all_input_ids=greek_all_input_ids,
    all_attention_masks=greek_all_attention_masks,
    all_sentences_tokens = greek_all_sentences_tokens,
    all_origin_indices=greek_all_origin_indices,
    all_subtoken_lemmas=greek_all_subtoken_lemmas
)

Processing batches: 100%|██████████| 5642/5642 [27:20<00:00,  3.44it/s]


In [ ]:
philo_latin_embeddings = get_contextual_embeddings_with_weights(
    processed_texts=filtered_lat_corpus,
    model=model,
    tokenizer=tokenizer,
    device=device,
    all_input_ids=latin_all_input_ids,
    all_attention_masks=latin_all_attention_masks,
    all_sentences_tokens = latin_all_sentences_tokens,
    all_origin_indices=latin_all_origin_indices,
    all_subtoken_lemmas=latin_all_subtoken_lemmas
)

Processing batches: 100%|██████████| 999/999 [08:28<00:00,  1.96it/s]


In [ ]:
import random

random_lemmas = random.sample(list(philo_greek_embeddings.keys()), 10)
for lemma in random_lemmas:
    print(lemma, philo_greek_embeddings[lemma][:5])

ἀγρίαι [ 0.02496342  0.02377088 -0.00303145 -0.06161425 -0.08300912]
διομολογητής [-0.02923197  0.02017769  0.09088974  0.03340148 -0.01367698]
λυτέω [-0.09446092  0.1343998  -0.03786481 -0.08447126 -0.03322066]
Ῥήγιον [ 0.10592197  0.03958572 -0.02032054 -0.0895769   0.12957276]
ἥντινά [-0.1520741  -0.09370718 -0.02544442  0.01203235  0.03966599]
δίμηνος [ 0.08202566 -0.0320841   0.0182681   0.0709415   0.11611687]
κυμαίνω [-0.08296876 -0.00725024  0.03758634 -0.02565242  0.04502526]
Θεόγενος [ 0.03436761  0.03748453  0.17158794 -0.16107477  0.12947322]
πηφίζω [-0.03666924  0.00672445  0.08354523 -0.07866172  0.0521444 ]
—καθάπερ [-0.09404872 -0.08564957  0.04148288 -0.19891717  0.06261841]


In [ ]:
with open('/content/drive/MyDrive/embeddings_save/philo_greek_embeddings.pkl', 'wb') as file:
    pickle.dump(philo_greek_embeddings, file)

In [ ]:
!gdown 1-6hW1nioaIYSSOemsdBdwSLqfHrYyLTV

In [ ]:
with open("/content/drive/MyDrive/embeddings_save/philo_greek_embeddings.pkl", "rb") as f:
    philo_greek_embeddings = pickle.load(f)

In [ ]:
!gdown 1-8KV-B6X2iDd7GhGzcNhRtpVLssZwg0Q

In [ ]:
with open("/content/drive/MyDrive/embeddings_save/philo_latin_embeddings.pkl", "rb") as f:
    philo_latin_embeddings = pickle.load(f)

In [ ]:
# Afficher les dix premières clés des embeddings grecs
first_ten_keys = list(philo_greek_embeddings.keys())[:10]
print(first_ten_keys)

['ὁ', 'μέν', 'ἐν', 'Δῆλος', 'παρά', 'θεός', 'αὐτός', 'γνώμη', 'ἀποφήνω', 'συγγράφω']


In [ ]:
!gdown --id 1MZ4ld8j30ye3YGYy-T7V3Cyy1c4dlGks

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1MZ4ld8j30ye3YGYy-T7V3Cyy1c4dlGks
To: /content/stopwords_gk.txt
100% 98.3k/98.3k [00:00<00:00, 136MB/s]


In [ ]:
!gdown --id 161g7Kdv4PCFp2iYJAlMbHkb-CGstsNcu

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=161g7Kdv4PCFp2iYJAlMbHkb-CGstsNcu
To: /content/stopwords_lat.txt
100% 35.4k/35.4k [00:00<00:00, 93.1MB/s]


In [ ]:
def remove_diacritics(char):
    normalized_char = unicodedata.normalize('NFD', char)
    non_diacritic_char = ''.join(c for c in normalized_char if not unicodedata.combining(c))
    return non_diacritic_char

In [ ]:
with open("/content/stopwords_gk.txt", encoding = "utf8") as stop_file:
  stopwords_diacritics=stop_file.read().split("\n")
stopwords_greek=set(stopwords_diacritics)

In [ ]:
with open("/content/stopwords_lat.txt", encoding = "utf8") as stop_file:
  stopwords_diacritics=stop_file.read().split("\n")
stopwords_latin=set(stopwords_diacritics)

In [ ]:
no_stop_greek_embeddings = {k: v for k, v in philo_greek_embeddings.items() if k not in stopwords_greek}

In [ ]:
first_ten_keys = list(no_stop_greek_embeddings.keys())[:10]
print(first_ten_keys)

['Δῆλος', 'θεός', 'γνώμη', 'ἀποφήνω', 'συγγράφω', 'προπύλαιος', 'Λητόος', 'διαιρέω', 'ὑπάρχω', 'ἀγαθός']


In [ ]:
no_stop_latin_embeddings = {k: v for k, v in philo_latin_embeddings.items() if k not in stopwords_latin}

In [ ]:
len(no_stop_greek_embeddings.keys())

349488

In [ ]:
from collections import Counter

# 1. Calculer la fréquence des lemmes
frequency_counter = Counter()
for filename, lemmatized_sentences in filtered_grc_corpus["lemmas"].items():  # Attention ici, j'ai ajouté ["lemmas"]
    for lemmatized_text in lemmatized_sentences:
        for lemma in lemmatized_text:
            frequency_counter[lemma] += 1

# 2. Trier les lemmes par fréquence
sorted_lemmas = [item[0] for item in frequency_counter.most_common()]

# 3. Sélectionner les top N lemmes
top_greek_lemmas = set(sorted_lemmas[:10000])

# 4. Filtrer les embeddings
greek_filtered_embeddings = {lemma: no_stop_greek_embeddings[lemma] for lemma in top_greek_lemmas if lemma in no_stop_greek_embeddings}



In [ ]:
# 1. Calculer la fréquence des lemmes
frequency_counter = Counter()
for filename, lemmatized_sentences in filtered_lat_corpus["lemmas"].items():  # Attention ici, j'ai ajouté ["lemmas"]
    for lemmatized_text in lemmatized_sentences:
        for lemma in lemmatized_text:
            frequency_counter[lemma] += 1

# 2. Trier les lemmes par fréquence
sorted_lemmas = [item[0] for item in frequency_counter.most_common()]

# 3. Sélectionner les top N lemmes
top_latin_lemmas = set(sorted_lemmas[:10000])

# 4. Filtrer les embeddings
latin_filtered_embeddings = {lemma: no_stop_latin_embeddings[lemma] for lemma in top_latin_lemmas if lemma in no_stop_latin_embeddings}


In [ ]:
matching_greek_lemmas = [lemma for lemma in top_greek_lemmas if lemma in no_stop_greek_embeddings]
print(len(matching_greek_lemmas))

9623


In [ ]:
matching_latin_lemmas = [lemma for lemma in top_latin_lemmas if lemma in no_stop_latin_embeddings]
print(len(matching_latin_lemmas))

9571


In [ ]:
missing_lemmas = top_greek_lemmas - set(no_stop_greek_embeddings.keys())
print(missing_lemmas)

{'ἄν', 'σοῖς', 'τότε', 'ἔχω', 'ἅπαξ', 'λίαν', 'ἅμα', 'αὐτήν', 'γίγνομαι', 'αὐτόθι', 'ἀμφοτέρων', 'ποῖος', 'οὔτι', 'πλήν', 'μοί', 'εἰμί', 'ν', 'καθά', 'δεῦρο', 'τοιοῦτος', 'τοιόσδε', 'αὔτως', 'ἔοικα', 'ἔπειτ', 'ἕως', 'γάρ', 'ἕτερος', 'γοῦν', 'τἆλλα', 'πού', 'πάν', 'πω', 'μήθ', 'κἀκεῖνα', 'βαίνω', 'νυν', 'τε', 'δ', 'ἡμᾶς', 'ὑμέτερος', 'ἡμέτερος', 'μάλα', 'εἶτα', 'οὐδέποτε', 'ἔστε', 'ἐπειδάν', 'ἔξωθεν', 'ὡσαύτως', 'ἔρχομαι', 'εὐθύς', 'μού', 'ἐγώ', 'ἡνίκα', 'ἀμφότεροι', 'ἐπήν', 'ποιέω', 'πάρος', 'ἄνωθεν', 'πᾶς', 'εὖ', 'λ', 'σεῦ', 'περί', 'ἔνι', 'διά', 'μηκέτι', 'παρά', 'πάντως', 'ὅπερ', 'οὗ', 'εἶμι', 'ἀμφοῖν', 'ἔοι', 'κἀν', 'καθάπερ', 'ταὐτός', 'οὐδείς', 'προσέτι', 'εἰμι', 'ἄλλοθεν', 'ἐπειδή', 'α', 'ἀλλήλων', 'αὕτη', 'ἔξω', 'νυνδή', 'οὖν', 'πότερον', 'ἔφη', 'ἑτέρως', 'ἤδη', 'εἴσω', 'ἐν', 'ὁ', 'ἀπό', 'αὐτάρ', 'ἐκεῖθεν', 'πάντων', 'οὗτος', '.', 'πότερος', 'εὖτε', 'ἄλλος', 'σύ', 'ξύν', 'οὔκουν', 'ἆρα', 'ἐκτός', 'ἐγγύς', 'ὡς', 'διό', 'ἅτε', 'ἠύτε', 'εἰ', 'ὅπως', 'καίτοι', 'ὁπότε', 'καθόλου', '

In [ ]:
# Extract the lemmas and their embeddings
all_embeddings = {**latin_filtered_embeddings, **greek_filtered_embeddings}
# Save embeddings to the vectors file
# Écrivez le fichier de vecteurs
with open('vectors.tsv', 'w') as f_vectors:
    for lemma, embedding in all_embeddings.items():
        f_vectors.write('\t'.join([str(value) for value in embedding]) + '\n')

# Écrivez le fichier de métadonnées
with open('metadata.tsv', 'w') as f_metadata:
    for lemma in all_embeddings.keys():
        f_metadata.write(lemma + '\n')

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def find_most_similar(target_embedding, embeddings_dict):
    similarities = {}
    for word, embedding in embeddings_dict.items():
        sim = cosine_similarity([target_embedding], [embedding])[0][0]
        similarities[word] = sim

    # Triez les mots par similarité de façon décroissante et prenez les 10 premiers
    sorted_items = sorted(similarities.items(), key=lambda x: x[1], reverse=True)

    return sorted_items[:10]

# À ce stade, la matrice `similarities` contient la similarité cosinus entre chaque paire d'embeddings.
# Par exemple, similarities[i, j] est la similarité entre le i-ème embedding latin et le j-ème embedding grec.


In [ ]:
greek_word = "λόγος"
greek_embedding = greek_filtered_embeddings[greek_word]  # Assurez-vous que cela est bien indexé

top_latin_words_with_scores = find_most_similar(greek_embedding, latin_filtered_embeddings)

for word, score in top_latin_words_with_scores:
    print(f"{word}: {score:.4f}")

oratio: 0.9894
τῷ: 0.9892
narratio: 0.9885
καὶ: 0.9878
liber: 0.9877
finis: 0.9877
scribo: 0.9877
loquor: 0.9875
ratio: 0.9875
argumentum: 0.9872
